### 0.0 follow setup instructions

ℹ️ use [`pylcaio.yml`](https://github.com/michaelweinold/config_conda/blob/main/pylcaio.yml) to set up working conda environment.

### 0.1. imports
#### 0.1.1. regular imports

In [64]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
import git
import json
# os specific settings
import platform
# configuration
import yaml
# lca
import ecospold2matrix as e2m
import pymrio
#import brightway2 as bw
# type hints
from ecospold2matrix import ecospold2matrix
from pymrio import IOSystem
# data science
import pandas as pd
import numpy as np
# deep copy
import copy

#### 0.1.2. load configuration file

In [2]:
with open('../config.yaml', 'r') as filestream:
    config = yaml.load(filestream, Loader = yaml.FullLoader)

#### 0.1.3. load `pylcaio`

In [3]:
sys.path.append(os.path.join(Path.home(), config['pylcaio'])) # required for local import of pylcaio
import pylcaio

### 0.2. file paths
#### 0.2.1. directories

In [7]:
%%capture
# home directory
print(path_dir_home := Path.home())
print(path_dir_repo := git.Repo('.', search_parent_directories=True).working_tree_dir)
# input directory
print(path_dir_databases := os.path.join(path_dir_home, config['path_dir_databases']))
# output directories
print(path_dir_data := os.path.join(path_dir_home, config['path_dir_data']))
print(path_dir_pylcaio := os.path.join(path_dir_home, path_dir_data, config['path_dir_pylcaio']))
print(path_dir_pymrio := os.path.join(path_dir_home, path_dir_data, config['path_dir_pymrio']))
print(path_dir_e2m := os.path.join(path_dir_home, path_dir_data, config['path_dir_e2m']))

#### 0.2.2. files

In [8]:
%%capture
# databases
print(path_exiobase := os.path.join(path_dir_home, path_dir_databases, config['exiobase']))
print(path_dir_ecoinvent := os.path.join(path_dir_home, path_dir_databases, config['ecoinvent']))
# pylcaio output
print(path_pylcaio_database_loader_class_instance := os.path.join(path_dir_pylcaio, config['pylcaio_database_loader_class_instance']))
print(path_pylcaio_class_instance_before_hybrid := os.path.join(path_dir_pylcaio, config['pylcaio_class_instance_before_hybrid']))
print(path_pylcaio_class_instance_after_hybrid := os.path.join(path_dir_pylcaio, config['pylcaio_class_instance_after_hybrid']))
# pymrio output
print(path_pymrio_class_instance := os.path.join(path_dir_pymrio, config['pymrio_class_instance']))
# e2m output
print(e2m_project_name := config['e2m_project_name'])
print(path_file_e2m_pickle := os.path.join(path_dir_e2m, e2m_project_name + config['e2m_pickle_filename']))

In [11]:
%%capture
print(path_dict_io_countries_per_lca_region := os.path.join(path_dir_repo, config['path_dict_io_countries_per_lca_region']))
print(path_list_io_countries_and_regions := os.path.join(path_dir_repo, config['path_list_io_countries_and_regions']))

In [14]:
with open(file = path_dict_io_countries_per_lca_region, mode = 'r', encoding = 'utf-8') as filestream:
    dict_io_countries_per_lca_region: dict = json.load(fp = filestream)
with open(file = path_list_io_countries_and_regions, mode = 'r', encoding = 'utf-8') as filestream:
    list_io_countries_and_regions: list = json.load(fp = filestream)

In [17]:
with open(path_pylcaio_class_instance_before_hybrid, 'rb') as file_in:
    pylcaio_object_before_hybrid: pylcaio.LCAIO = pd.read_pickle(file_in)

#### 1.1. function implementation

PRO_f dataframe:

| index | activityNameId | io_geography |
| ----- | -------------- | ------------ |
| 10 | 1 | RoW |
| 11 | 1 | CH |
| 12 | 1 | AT |
| 13 | 2 | RoW |
| 14 | 2 | DE |
| 15 | 2 | CH |
| 16 | 3 | FR |
| 17 | 3 | BE |
| 18 | 4 | RoW |
| 19 | 4 | CH |
| 20 | 4 | AT |

should look like:

| activityNameId | io_geography_list | RoW_region |
| -------------- | ----------------- | ---------- |
| 1 | RoW, CH, AT | RoW(1) |
| 2 | RoW, DE, CH | RoW(2) |
| 4 | RoW, CH, AT | RoW(1) |

where for activityNameId == 1, RoW region is list_io_countries - [CH, AT]


check: https://stackoverflow.com/a/53343046

In [33]:
# smart implementation

def identify_rest_of_world_regions(
    df: pd.DataFrame,
    list_io_countries_and_regions: list,
    dict_io_countries_per_lca_region: dict
) -> pd.DataFrame:
    """
    bananas!

    Args:
        df (pd.DataFrame): _description_
        list_io_countries_and_regions (list): _description_
        dict_io_countries_per_lca_region (dict): _description_

    Returns:
        pd.DataFrame: _description_
    """

    df_aggregated = pd.DataFrame(
        data = df.groupby('activityNameId')['io_geography'].apply(tuple)
    )
    
    df_aggregated: pd.DataFrame(
        data =  df_aggregated['io_geography'].apply(
            lambda x: bool(set(x) & set(['RoW']))
        )
    )

    unique_rest_of_world_regions = pd.DataFrame(df['io_geography'].unique())
    unique_rest_of_world_regions['rest_of_world_index'] = ['RoW({})'.format(i) for i in unique_rest_of_world_regions.index]

    df_aggregated.merge(
        right = unique_rest_of_world_regions,
        how = 'left',
        on = 'io_geography'
    )

    # remove 'RoW'

    df[df['list_io_geography'].isin['RoW']]

    return df

In [59]:
df = pd.DataFrame(
        data = PRO_f.groupby('activityNameId')['io_geography'].apply(tuple)
    )

In [68]:
testdf = pd.DataFrame(df['io_geography'].unique())

In [75]:
testdf.index()

TypeError: 'RangeIndex' object is not callable

In [80]:
testdf['ind'] = ['RoW({})'.format(i) for i in testdf.index]

In [81]:
testdf

,0,ind
0,"(GLO,)",RoW(0)
1,"(HR, MX, WA, LT, HR, LT, WA, RU, ID, WL, LV, W...",RoW(1)
2,"(CH, RoW, Europe without Switzerland)",RoW(2)
3,"(RER, RoW)",RoW(3)
4,"(BR, BR, US, US, RER, RoW, RoW, RER)",RoW(4)
...,...,...
539,"(WE, WE, WE)",RoW(539)
540,"(UCTE without Germany, RoW, DE)",RoW(540)
541,"(WE, ES, RO, SI, SE, GB, FI, GR, AT, PL, CY, B...",RoW(541)
542,"(NO, WM, CN, FI, GB, IT, EE, CN, KR, JP, WE, B...",RoW(542)


In [66]:
test: np.ndarray[list] = df['io_geography'].unique()

In [67]:
test

array([('GLO',),
       ('HR', 'MX', 'WA', 'LT', 'HR', 'LT', 'WA', 'RU', 'ID', 'WL', 'LV', 'WL', 'TW', 'ZA', 'BG', 'MX', 'BG', 'RoW', 'WE', 'RU', 'WA', 'EE', 'WL', 'ID', 'LV', 'TR', 'ZA', 'TR', 'CH', 'WF', 'CH', 'EE', 'WA', 'TW', 'WF', 'RoW', 'WE', 'WL'),
       ('CH', 'RoW', 'Europe without Switzerland'), ('RER', 'RoW'),
       ('BR', 'BR', 'US', 'US', 'RER', 'RoW', 'RoW', 'RER'),
       ('RoW', 'WF', 'RU', 'NO', 'NL', 'DE'), ('CH', 'RoW'),
       ('CN', 'CN', 'BE', 'DE', 'IE', 'RoW', 'GB', 'FR', 'CN', 'JP', 'DK', 'ES', 'KR', 'EE', 'SE', 'CN', 'FI', 'PL', 'NL', 'CN', 'NO', 'PT'),
       ('RoW', 'RER'), ('RoW', 'CH', 'Europe without Switzerland', 'US'),
       ('CA',),
       ('CA', 'CA', 'RoW', 'RER', 'RoW', 'CA', 'RoW', 'RER', 'RER'),
       ('CA', 'US', 'CA'), ('DE', 'RoW', 'RER w/o CH+DE', 'CH'),
       ('RoW', 'CH', 'CA'),
       ('AU', 'RoW', 'AU', 'RNA', 'RLA', 'RER', 'RAS', 'RER', 'RoW', 'RNA', 'RAS', 'RLA'),
       ('Europe without Switzerland', 'RoW', 'CH'),
       ('US', 'CH

In [32]:
PRO_f['list_geo']

a96cb241-a4a9-4980-a16a-ba4b6a80175e_aeaf5266-3f9c-4074-bd34-eba76a61760c    NaN
6885fd40-ff73-40a4-8f71-225577ec684e_aeaf5266-3f9c-4074-bd34-eba76a61760c    NaN
d1c3b8ad-58b6-4ea4-8877-a51b356556bc_281fc4f0-c05d-410a-a784-06e3508e78e6    NaN
92068396-88c7-45ed-9008-622008a299f3_0d860eb4-1a25-41b4-a821-81f5726d86e5    NaN
35ddb020-9812-4808-bdfb-6845a454a73c_0d860eb4-1a25-41b4-a821-81f5726d86e5    NaN
                                                                            ... 
31e6ab74-141d-49f4-9e75-5974bd62742b_b3184435-d00c-5713-823d-5ff6741bfbbc    NaN
b50b032c-8bf0-41a4-bb7a-ce5e12cabf01_d13ae29b-7d6a-55d2-8380-86343a80fed8    NaN
307b681d-c6bc-44cb-ab3e-a9067a4e115a_ed25f589-c6c8-56a7-a518-b01dd3e8a453    NaN
73c9e29c-885e-490e-95f3-087227443758_101e55f2-2da0-528b-96fd-21687e242c1c    NaN
91ed532a-08f2-4fdd-b5e9-28bb2afd1336_451550a8-d9e6-4396-9f9a-97e752111228    NaN
Name: list_geo, Length: 16022, dtype: object

In [36]:
df_aggregated = pd.DataFrame(
        data = PRO_f.groupby('activityNameId')['io_geography'].apply(list)
    )

In [52]:
df_aggregated[df_aggregated['io_geography'].apply(lambda x: bool(set(x) & set(['RoW'])))]

,io_geography
activityNameId,
003682ba-f70c-474f-ba5f-8203d2d00512,"[HR, MX, WA, LT, HR, LT, WA, RU, ID, WL, LV, W..."
00414c23-6514-4699-9d90-71e9cf9a0262,"[CH, RoW, Europe without Switzerland]"
00431801-ae3e-4836-aca5-838071f41957,"[RER, RoW]"
0043d228-ff21-4697-9e99-897c9bb4a6e5,"[BR, BR, US, US, RER, RoW, RoW, RER]"
0055a295-7c95-46c1-8baa-1a9bf2a364b5,"[RoW, WF, RU, NO, NL, DE]"
...,...
ff7f2df7-3797-40f0-bef5-2de2b6c1f4ef,"[CH, RoW]"
ff98d350-178d-463c-873a-e78a2be05336,"[RoW, RER]"
ffc05716-a13b-4d62-a9ea-b5ca4a738945,"[RoW, RER]"


In [45]:
df_aggregated['io_geography']

activityNameId
0021a538-9534-4fd4-835e-a2fa8f76bb9d                                                [GLO]
002477cd-70fa-49c1-afc3-c372abbcf72b                                                [GLO]
002fa54c-1829-4231-82ed-47e35a6745d4                                                [GLO]
003682ba-f70c-474f-ba5f-8203d2d00512    [HR, MX, WA, LT, HR, LT, WA, RU, ID, WL, LV, W...
003a2fba-a774-4ff8-b8da-3adb2e14aeba                                                [GLO]
                                                              ...                        
ffb6f7f9-766a-40b8-9401-8ffb6599310b                                                [GLO]
ffc05716-a13b-4d62-a9ea-b5ca4a738945                                           [RoW, RER]
ffd2d364-1641-4b66-a350-455eff193e37                            [RoW, CH, RU, DE, FR, US]
ffe2f2d1-4266-4992-b0e6-836c27fb8457                                                [GLO]
fff34601-b48b-43ea-89da-3e33d8f4df5a                                           [RoW, 

In [49]:
df = df_aggregated

In [50]:
df[df['io_geography'].isin(['GLO'])]

,io_geography
activityNameId,


In [43]:
df_aggregated[df_aggregated['io_geography'].str.contains('GLO').any(axis='columns').values]


ValueError: No axis named columns for object type Series

In [18]:
PRO_f = pylcaio_object_before_hybrid.PRO_f

In [20]:
PRO_f.columns

Index(['activityId', 'productId', 'activityName', 'ISIC', 'price', 'priceUnit',
       'EcoSpoldCategory', 'geography', 'technologyLevel',
       'macroEconomicScenario', 'wet mass [kg]_x', 'dry mass [kg]_x',
       'productionVolume', 'productName', 'unitName', 'cpc', 'dry mass [kg]_y',
       'wet mass [kg]_y', 'activityNameId', 'activityType', 'startDate',
       'endDate', 'activityName_duplicate', 'io_geography', 'ProductTypeName'],
      dtype='object')

In [28]:
identify_rest_of_world_regions(
    df = PRO_f,
    list_io_countries_and_regions = list_io_countries_and_regions,
    dict_io_countries_per_lca_region = dict_io_countries_per_lca_region
)

,activityId,productId,activityName,ISIC,price,priceUnit,EcoSpoldCategory,geography,technologyLevel,macroEconomicScenario,...,dry mass [kg]_y,wet mass [kg]_y,activityNameId,activityType,startDate,endDate,activityName_duplicate,io_geography,ProductTypeName,list_io_geography
a96cb241-a4a9-4980-a16a-ba4b6a80175e_aeaf5266-3f9c-4074-bd34-eba76a61760c,a96cb241-a4a9-4980-a16a-ba4b6a80175e,aeaf5266-3f9c-4074-bd34-eba76a61760c,"barley grain, feed production","0111:Growing of cereals (except rice), legumin...",0.124,EUR2005,agricultural means of production/feed,CA-QC,Current,Business-as-Usual,...,0.86,1,2816d5b0-50fd-42fd-9bc9-aed0bd1913e7,0,1996-01-01,2018-12-31,"barley grain, feed production",CA,Cereal grains nec,NaN
6885fd40-ff73-40a4-8f71-225577ec684e_aeaf5266-3f9c-4074-bd34-eba76a61760c,6885fd40-ff73-40a4-8f71-225577ec684e,aeaf5266-3f9c-4074-bd34-eba76a61760c,"barley grain, feed production","0111:Growing of cereals (except rice), legumin...",0.124,EUR2005,agricultural means of production/feed,RoW,Current,Business-as-Usual,...,0.86,1,2816d5b0-50fd-42fd-9bc9-aed0bd1913e7,0,1996-01-01,2018-12-31,"barley grain, feed production",RoW,Cereal grains nec,NaN
d1c3b8ad-58b6-4ea4-8877-a51b356556bc_281fc4f0-c05d-410a-a784-06e3508e78e6,d1c3b8ad-58b6-4ea4-8877-a51b356556bc,281fc4f0-c05d-410a-a784-06e3508e78e6,"barley grain, feed production, Swiss integrate...","0111:Growing of cereals (except rice), legumin...",0.124,EUR2005,agricultural means of production/feed,CH,Current,Business-as-Usual,...,0.85,1,938a1c4e-41f2-40ea-9bf1-9f09c2437798,0,1996-01-01,2018-12-31,"barley grain, feed production, Swiss integrate...",CH,Cereal grains nec,NaN
92068396-88c7-45ed-9008-622008a299f3_0d860eb4-1a25-41b4-a821-81f5726d86e5,92068396-88c7-45ed-9008-622008a299f3,0d860eb4-1a25-41b4-a821-81f5726d86e5,"barley grain, feed production, organic","0111:Growing of cereals (except rice), legumin...",0.159,EUR2005,agricultural means of production/feed,CH,Current,Business-as-Usual,...,0.85,1,6a49a0c6-c114-4471-b6c4-8aa4006e4b0e,0,1996-01-01,2018-12-31,"barley grain, feed production, organic",CH,Cereal grains nec,NaN
35ddb020-9812-4808-bdfb-6845a454a73c_0d860eb4-1a25-41b4-a821-81f5726d86e5,35ddb020-9812-4808-bdfb-6845a454a73c,0d860eb4-1a25-41b4-a821-81f5726d86e5,"barley grain, feed production, organic","0111:Growing of cereals (except rice), legumin...",0.159,EUR2005,agricultural means of production/feed,RoW,Current,Business-as-Usual,...,0.85,1,6a49a0c6-c114-4471-b6c4-8aa4006e4b0e,0,1996-01-01,2018-12-31,"barley grain, feed production, organic",RoW,Cereal grains nec,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31e6ab74-141d-49f4-9e75-5974bd62742b_b3184435-d00c-5713-823d-5ff6741bfbbc,31e6ab74-141d-49f4-9e75-5974bd62742b,b3184435-d00c-5713-823d-5ff6741bfbbc,"waste polyethylene terephthalate, for recyclin...",NaN,0.000,NaN,NaN,GLO,Current,Business-as-Usual,...,NaN,NaN,712ffc38-2281-4fd2-a84d-089678e23bd3,0,2018-01-01,2018-12-31,"waste polyethylene terephthalate, for recyclin...",GLO,NaN,NaN
b50b032c-8bf0-41a4-bb7a-ce5e12cabf01_d13ae29b-7d6a-55d2-8380-86343a80fed8,b50b032c-8bf0-41a4-bb7a-ce5e12cabf01,d13ae29b-7d6a-55d2-8380-86343a80fed8,"waste polyethylene terephthalate, for recyclin...",NaN,0.000,NaN,NaN,GLO,Current,Business-as-Usual,...,NaN,NaN,27dcd766-bc63-4a7c-9fdf-ffc55e0575b2,0,2018-01-01,2018-12-31,"waste polyethylene terephthalate, for recyclin...",GLO,NaN,NaN
307b681d-c6bc-44cb-ab3e-a9067a4e115a_ed25f589-c6c8-56a7-a518-b01dd3e8a453,307b681d-c6bc-44cb-ab3e-a9067a4e115a,ed25f589-c6c8-56a7-a518-b01dd3e8a453,"waste polyethylene, for recycling, sorted, Rec...",NaN,0.000,NaN,NaN,GLO,Current,Business-as-Usual,...,NaN,NaN,40de0860-eadc-43ae-bdbf-5e507563379a,0,2018-01-01,2018-12-31,"waste polyethylene, for recycling, sorted, Rec...",GLO,NaN,NaN
73c9e29c-885e-490e-95f3-087227443758_101e55f2-2da0-528b-96fd-21687e242c1c,73c9e29c-885e-490e-95f3-087227443758,101e55f2-2da0-528b-96fd-21687e242c1c,"waste polyethylene, for recycling, unsorte

In [ ]:
def identify_rows(self):
    """ Method to identify the various unique Rest of the World (RoW) regions of the LCA database

    Returns:
    --------
        The updated self.dictRoW in which unique RoW regions are identified in terms of countries they include

    """

    # contains a list of activityNameId's that are 'PRO_f.io_geography[i]=='RoW (=LCA RoW)'
    unique_activities_using_row = list(
        set( # removes duplicates
            self.PRO_f.activityNameId[ # nota bene: activityNameId is Master Data!
                [i for i in self.PRO_f.index if self.PRO_f.io_geography[i] == 'RoW']].tolist()
        )
    )

    RoW_activities = defaultdict(list)

    # how and WHERE are `geography` and `io_geography` matched?

    # list of tuples (activityNameId (=Master Data), geography)
    # where activityNameId where one of the geographis is 'RoW' and
    # where geography is a list of all regions that are associated with one activityNameId
    tupl = [  
        i for i in zip( # zip creates tuple
            self.PRO_f.activityNameId.loc[
                [
                    i for i in self.PRO_f.index
                    if self.PRO_f.activityNameId[i] in unique_activities_using_row
                ]
            ],
            self.PRO_f.io_geography.loc[
                [
                    i for i in self.PRO_f.index
                    if self.PRO_f.activityNameId[i] in unique_activities_using_row
                ]
            ]
        )
    ]

    # ok, dictionary is filled
    for activity, geography in tupl:
        RoW_activities[activity].append(geography)

    # remove 'RoW' from dict values
    RoW_activities = {activity: [geography1 for geography1 in geography if geography1 != 'RoW'] for activity, geography in RoW_activities.items()}

    # delete from RoW_activities processes which had only RoW as geography and are thus empty now
    for key in [i for i in list(RoW_activities.keys()) if RoW_activities[i] == []]:
        del RoW_activities[key]
        
    # put every element to the same level (elements that are lists are transformed to lists of lists)
    for values in list(RoW_activities.values()):
        for i in range(0, len(values)):
            if values[i] in self.countries_per_regions.keys():
                values[i] = self.countries_per_regions[values[i]]

    # for elements that are lists of lists stemming from the replacement of ['RER'] by [['AT','BE',...]],
    # add all of the together in a single list
    for keys in RoW_activities.keys():
        for item in RoW_activities[keys]:
            if isinstance(item, list):
                RoW_activities[keys] = sum_elements_list(RoW_activities[keys])
    # remove duplicates inside the elements
    for keys in list(RoW_activities.keys()):
        RoW_activities[keys] = list(set(RoW_activities[keys]))

    # why sort here? to ensure unique_RoWs does not contain duplicates that would just have a different order
    # need to sort to identify duplicates whose elements would be ordered differently and thus be treated as not duplicated
    for keys in RoW_activities.keys():
        RoW_activities[keys].sort()

    # identify the combination of countries that are NOT inside the residual of each process
    # dict where
    # key = activityNameId
    # value = list of IO countries that are NOT associated with activityNameId
    dictactrow = {}
    residual_geo_IO_to_remove = ['WA', 'WE', 'WF', 'WL', 'WM'] # exiobase 'rest of world' regions (all other are actual countries)
    for keys in RoW_activities.keys():
        dictactrow[keys] = list( # dictactrow[keys] is activityNameId
            set(self.listcountry) # countries + RoW regions IO
            - set(RoW_activities[keys]) # countries LCA associated with the activityNameId
            - set(residual_geo_IO_to_remove)) # RoW regions IO
    
    unique_RoWs = []
    for keys in dictactrow.keys():
        if dictactrow[keys] not in unique_RoWs:
            unique_RoWs.append(dictactrow[keys])

    # create name for the values of the different RoW
    listname = []
    for i in range(0, len(unique_RoWs)):
        listname.append('RoW' + '(' + str(i) + ')')
    
    # put all of that in dictRoW
    # dictRow is created empty earlier in the code
    for i in range(0, len(unique_RoWs)):
        self.dictRoW[listname[i]] = unique_RoWs[i]
    try:
        # if RoWs are empty because processes from ecoinvent are too described
        del [[k for k in self.dictRoW.keys() if len(self.dictRoW[k]) == 0][0]]
    except IndexError:
        pass

    # replace RoW list (eg. [AT, DE, ...] with RoW number (eg. RoW(1)))
    for activityNameId in dictactrow: # key = activityNameId
        for RoW_number in self.dictRoW: # key = eg. "RoW(12)"
            if dictactrow[activityNameId] == self.dictRoW[RoW_number]: # dictactrow[activityNameId] = RoW list, self.dictRoW[RoW_number]
                dictactrow[activityNameId] = RoW_number

    RoW_matrix = pd.DataFrame(
        data = list(dictactrow.values()),
        index=list(dictactrow.keys()),
        columns=['RoW_geography']
    )

    # adds RoW information to matrix
    self.PRO_f = self.PRO_f.merge(RoW_matrix, left_on='activityNameId', right_on=RoW_matrix.index, how='outer')

    self.PRO_f.index = self.PRO_f.activityId + '_' + self.PRO_f.productId
    self.PRO_f = self.PRO_f.reindex(self.processes_in_order)
    
    self.PRO_f.io_geography.update(self.PRO_f.RoW_geography[self.PRO_f.io_geography == 'RoW'])
    self.PRO_f = self.PRO_f.drop('RoW_geography', axis=1)

    # might be some RoW or empty lists left in PRO_f
    self.PRO_f.io_geography[self.PRO_f.io_geography == 'RoW'] = 'GLO'
    self.PRO_f.io_geography.loc[[i for i in self.PRO_f.index if type(self.PRO_f.io_geography[i]) == list]] = 'GLO'